In [373]:
import pandas as pd
import json
import mysql.connector
from sqlalchemy import create_engine
from sqlalchemy import URL

In [374]:
pd.options.display.max_columns = 100

In [375]:
username = 'root'
password='root'
host='localhost'
database='jmdb'

engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}/{database}")


In [376]:
df = pd.read_csv("emdb.csv",encoding="utf-16", sep = ";")

In [377]:
df = df.rename(columns = {v : v.strip() for v in df.columns})

In [378]:
print(df.columns)

Index(['Numéro', 'Titre', 'Titre alternatif', 'Année', 'Pays', 'Genre(s)',
       'Langues', 'Durée', 'Chemin de l'affiche', 'Réalisateur(s)',
       'Scénariste(s)', 'Compositeur(s)', 'Studio', 'Distribution',
       'Bande-annonce', 'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
       'Commentaires', 'Marqueurs', 'Support', 'Taille du fichier',
       'ID TheMovieDb', 'ID IMDb'],
      dtype='object')


# table movie , director et intermediaire

In [379]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [380]:
# Modifier le chemin de l'affiche pour ne conserver que la dernière partie
df_movie["Chemin de l'affiche"] = df_movie["Chemin de l'affiche"].astype(str).apply(lambda x: x.split('\\')[-1])

# Afficher le DataFrame résultant
print(df_movie.head())  # Affiche les premières lignes du DataFrame

   Numéro                           Titre  \
0    3904      Yokohama BJ blues (vostfr)   
1    3903               Vertiges (vostfr)   
2    3902                   Transcendance   
3    3901  Shadow Warriors: Hattori Hanzo   
4    3900                      Brainstorm   

                        Titre alternatif Année  Durée Chemin de l'affiche  \
0            ヨコハマＢＪブルース (Titre original)  1981    112          003988.jpg   
1  Per le antiche scale (Titre original)  1975    110          003987.jpg   
2         Transcendence (Titre original)  2014    119          003986.jpg   
3             影の軍団 服部半蔵 (Titre original)  1980    133          003985.jpg   
4                                    NaN  1983    106          003984.jpg   

                         Bande-annonce  \
0  www.youtube.com/watch?v=oYzkD-YJbjo   
1                                  NaN   
2  www.youtube.com/watch?v=wUhsUgd-HrY   
3                                  NaN   
4  www.youtube.com/watch?v=NNiZP2G-nEM   

             

In [381]:
list_director_movie = ["Réalisateur(s)", 'Numéro']
df_director_movie = df.copy()[list_director_movie]

In [382]:
# df_director_movie ["Réalisateur(s)"] = df_director_movie["Réalisateur(s)"].str.split(",")
df_director_movie["Réalisateur(s)"] = df_director_movie["Réalisateur(s)"].str.split(",\s*")

In [383]:
df_director_movie  = df_director_movie.explode("Réalisateur(s)")

In [384]:
df_director  = df_director_movie.explode("Réalisateur(s)").drop_duplicates("Réalisateur(s)").reset_index(drop=True).reset_index()
df_director = df_director.rename(columns={"index":"id"}).drop(columns="Numéro")

#.reset_index().rename(columns={"index":"id"})

In [385]:
df_intermediaire_movie_director = df_director.merge(df_director_movie, on = "Réalisateur(s)").drop(columns="Réalisateur(s)")

In [386]:
# Remplacer les valeurs NULL de la colonne 'id' par 0
df_director['id'] = df_director['id'].fillna(0)
# Convertir la colonne 'id' en type entier
df_director['id'] = df_director['id'].astype(int)

# Réorganiser les ID si nécessaire pour éviter les conflits
max_id = df_director['id'].max()
if max_id > 0:
    df_director['id'] = df_director['id'] + 1

In [387]:
display(df_director)

,id,Réalisateur(s)
0,1,Eiichi Kudo
1,2,Mauro Bolognini
2,3,Wally Pfister
3,4,Douglas Trumbull
4,5,Chen Kuofu
...,...,...
1919,1920,Richard C. Sarafian
1920,1921,Chris Petit
1921,1922,Brett Ratner
1922,1923,Buster Keaton


In [388]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3904,Yokohama BJ blues (vostfr),ヨコハマＢＪブルース (Titre original),1981,112,003988.jpg,www.youtube.com/watch?v=oYzkD-YJbjo,NaN,BJ est un chanteur de blues relativement incon...,F:\#DIVX\DIVX MOVIEZ\POLAR\Yokohama BJ Blues (...,Matroska (MKV),NaN,Fichier multimédia,1.93 GB,movie/318955,83351.0
1,3903,Vertiges (vostfr),Per le antiche scale (Titre original),1975,110,003987.jpg,NaN,Dante's Inferno is Alive and Well,"En Italie, dans les années 1930, le professeur...",F:\#DIVX\DIVX MOVIEZ\DRAMA\Vertiges (1975) VOS...,AVI (Microsoft),NaN,Fichier multimédia,1.73 GB,movie/105529,73529.0
2,3902,Transcendance,Transcendence (Titre original),2014,119,003986.jpg,www.youtube.com/watch?v=wUhsUgd-HrY,Hier le Dr Will Caster n'était qu'humain...,Le Dr Will Caster est un spécialiste de l'IA t...,F:\#DIVX\DIVX MOVIEZ\MadMov\Transcendance -Wal...,Matroska (MKV),NaN,Fichier multimédia,2.61 GB,movie/157353,2209764.0
3,3901,Shadow Warriors: Hattori Hanzo,影の軍団 服部半蔵 (Titre original),1980,133,003985.jpg,NaN,Eiichi Kudo's strangest film that started the ...,"L'histoire se déroule vers 1650, après la mort...",F:\#DIVX\DIVX MOVIEZ\MadMov\Shadow Warriors - ...,Matroska (MKV),NaN,Fichier multimédia,996 MB,movie/208908,6975420.0
4,3900,Brainstorm,NaN,1983,106,003984.jpg,www.youtube.com/watch?v=NNiZP2G-nEM,... l'expérience ultime,"Lillian Reynolds et «Mike» Brace, réussissent ...",F:\#DIVX\DIVX MOVIEZ\MadMov\Brainstorm (1983) ...,Matroska (MKV),NaN,Fichier multimédia,5.92 GB,movie/15050,85271.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3899,5,le mécano de la général,The General (Titre original),1926,79,000005.jpg,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3900,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,000004.jpg,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3901,3,Les feux de la rampe,Limelight (Titre original),1952,137,000003.jpg,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3902,2,Les temps modernes,Modern Times (Titre original),1936,87,000002.jpg,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [389]:
display(df_intermediaire_movie_director)

,id,Numéro
0,0,3904
1,0,3901
2,0,3811
3,0,3510
4,1,3903
...,...,...
4213,1919,20
4214,1920,18
4215,1921,15
4216,1922,5


In [390]:
# Trouver l'index de la ligne avec le 'Réalisateur(s)' NULL
null_director_index = df_director[df_director['Réalisateur(s)'].isnull()].index

# Si une ligne avec 'Réalisateur(s)' NULL est trouvée
if not null_director_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_director.loc[null_director_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_director['id'].max()
    if max_id > 0:
        df_director.loc[df_director.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_director.head())

# Enregistrer le DataFrame modifié dans la table 'director' de la base de données
df_director.to_sql("director", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_director = df_director.merge(df_director_movie, on="Réalisateur(s)").drop(columns="Réalisateur(s)")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_director.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_director.to_sql("movie_director", index=False, con=engine, if_exists="replace")

     id    Réalisateur(s)
0  1925       Eiichi Kudo
1     2   Mauro Bolognini
2     3     Wally Pfister
3     4  Douglas Trumbull
4     5        Chen Kuofu
     id  Numéro
0  1925    3904
1  1925    3901
2  1925    3811
3  1925    3510
4     2    3903


4218

In [391]:
# df_director.to_sql("director",index=False,con = engine,if_exists = "replace" )

In [392]:
# df_intermediaire_movie_director.to_sql("movie_director",index=False,con = engine,if_exists = "replace" )

In [393]:
df_movie.to_sql("movies",index=False,con = engine,if_exists = "replace" )

3904

# table movie, screenwriter et intermediaire

In [394]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [395]:
list_screenwriter_movie = ["Scénariste(s)", 'Numéro']
df_screenwriter_movie = df.copy()[list_screenwriter_movie]

In [396]:
df_screenwriter_movie ["Scénariste(s)"] = df_screenwriter_movie["Scénariste(s)"].str.split(",\s*")

In [397]:
df_screenwriter_movie  = df_screenwriter_movie.explode("Scénariste(s)")

In [398]:
df_screenwriter  = df_screenwriter_movie.explode("Scénariste(s)").drop_duplicates("Scénariste(s)").reset_index(drop=True).reset_index()
df_screenwriter = df_screenwriter.rename(columns={"index":"id"}).drop(columns="Numéro")

In [399]:
df_intermediaire_movie_screenwriter = df_screenwriter.merge(df_screenwriter_movie, on = "Scénariste(s)").drop(columns="Scénariste(s)")

In [400]:
display(df_screenwriter)

,id,Scénariste(s)
0,0,Shōichi Maruyama
1,1,Mario Tobino
2,2,Tullio Pinelli
3,3,Jack Paglen
4,4,Yamada Takayuki
...,...,...
4145,4145,Ted Tally
4146,4146,David Veloz
4147,4147,Richard Rutowski
4148,4148,Buster Keaton


In [401]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3904,Yokohama BJ blues (vostfr),ヨコハマＢＪブルース (Titre original),1981,112,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=oYzkD-YJbjo,NaN,BJ est un chanteur de blues relativement incon...,F:\#DIVX\DIVX MOVIEZ\POLAR\Yokohama BJ Blues (...,Matroska (MKV),NaN,Fichier multimédia,1.93 GB,movie/318955,83351.0
1,3903,Vertiges (vostfr),Per le antiche scale (Titre original),1975,110,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,Dante's Inferno is Alive and Well,"En Italie, dans les années 1930, le professeur...",F:\#DIVX\DIVX MOVIEZ\DRAMA\Vertiges (1975) VOS...,AVI (Microsoft),NaN,Fichier multimédia,1.73 GB,movie/105529,73529.0
2,3902,Transcendance,Transcendence (Titre original),2014,119,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=wUhsUgd-HrY,Hier le Dr Will Caster n'était qu'humain...,Le Dr Will Caster est un spécialiste de l'IA t...,F:\#DIVX\DIVX MOVIEZ\MadMov\Transcendance -Wal...,Matroska (MKV),NaN,Fichier multimédia,2.61 GB,movie/157353,2209764.0
3,3901,Shadow Warriors: Hattori Hanzo,影の軍団 服部半蔵 (Titre original),1980,133,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,Eiichi Kudo's strangest film that started the ...,"L'histoire se déroule vers 1650, après la mort...",F:\#DIVX\DIVX MOVIEZ\MadMov\Shadow Warriors - ...,Matroska (MKV),NaN,Fichier multimédia,996 MB,movie/208908,6975420.0
4,3900,Brainstorm,NaN,1983,106,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=NNiZP2G-nEM,... l'expérience ultime,"Lillian Reynolds et «Mike» Brace, réussissent ...",F:\#DIVX\DIVX MOVIEZ\MadMov\Brainstorm (1983) ...,Matroska (MKV),NaN,Fichier multimédia,5.92 GB,movie/15050,85271.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3899,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3900,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3901,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3902,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [402]:
display(df_intermediaire_movie_screenwriter)

,id,Numéro
0,0,3904
1,1,3903
2,2,3903
3,2,2355
4,2,2303
...,...,...
7524,4145,15
7525,4146,14
7526,4147,14
7527,4148,5


In [403]:
# Trouver l'index de la ligne avec le 'Scénariste(s)' NULL
null_screenwriter_index = df_screenwriter[df_screenwriter['Scénariste(s)'].isnull()].index

# Si une ligne avec 'Scénariste(s)' NULL est trouvée
if not null_screenwriter_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_screenwriter.loc[null_screenwriter_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_screenwriter['id'].max()
    if max_id > 0:
        df_screenwriter.loc[df_screenwriter.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_screenwriter.head())

# Enregistrer le DataFrame modifié dans la table 'screenwriter' de la base de données
df_screenwriter.to_sql("screenwriter", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_screenwriter = df_screenwriter.merge(df_screenwriter_movie, on="Scénariste(s)").drop(columns="Scénariste(s)")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_screenwriter.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_screenwriter.to_sql("movie_screenwriter", index=False, con=engine, if_exists="replace")


     id     Scénariste(s)
0  4150  Shōichi Maruyama
1     1      Mario Tobino
2     2    Tullio Pinelli
3     3       Jack Paglen
4     4   Yamada Takayuki
     id  Numéro
0  4150    3904
1     1    3903
2     2    3903
3     2    2355
4     2    2303


7529

In [404]:
# df_screenwriter.to_sql("screenwriter",index=False,con = engine,if_exists = "replace" )

In [405]:
# df_intermediaire_movie_screenwriter.to_sql("movie_screenwriter",index=False,con = engine,if_exists = "replace" )

# table movie, music et intermediaire

In [406]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [407]:
list_music_movie = ["Compositeur(s)", 'Numéro']
df_music_movie = df.copy()[list_music_movie]

In [408]:
df_music_movie ["Compositeur(s)"] = df_music_movie["Compositeur(s)"].str.split(",\s*")

In [409]:
df_music_movie  = df_music_movie.explode("Compositeur(s)")

In [410]:
df_music  = df_music_movie.explode("Compositeur(s)").drop_duplicates("Compositeur(s)").reset_index(drop=True).reset_index()
df_music = df_music.rename(columns={"index":"id"}).drop(columns="Numéro")

In [411]:
df_intermediaire_movie_music = df_music.merge(df_music_movie, on = "Compositeur(s)").drop(columns="Compositeur(s)")

In [412]:
display(df_music)

,id,Compositeur(s)
0,0,Creation
1,1,Ennio Morricone
2,2,Mychael Danna
3,3,NaN
4,4,James Horner
...,...,...
1377,1377,Machine Head
1378,1378,Photek
1379,1379,John Frizzell
1380,1380,Leonard Cohen


In [413]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3904,Yokohama BJ blues (vostfr),ヨコハマＢＪブルース (Titre original),1981,112,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=oYzkD-YJbjo,NaN,BJ est un chanteur de blues relativement incon...,F:\#DIVX\DIVX MOVIEZ\POLAR\Yokohama BJ Blues (...,Matroska (MKV),NaN,Fichier multimédia,1.93 GB,movie/318955,83351.0
1,3903,Vertiges (vostfr),Per le antiche scale (Titre original),1975,110,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,Dante's Inferno is Alive and Well,"En Italie, dans les années 1930, le professeur...",F:\#DIVX\DIVX MOVIEZ\DRAMA\Vertiges (1975) VOS...,AVI (Microsoft),NaN,Fichier multimédia,1.73 GB,movie/105529,73529.0
2,3902,Transcendance,Transcendence (Titre original),2014,119,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=wUhsUgd-HrY,Hier le Dr Will Caster n'était qu'humain...,Le Dr Will Caster est un spécialiste de l'IA t...,F:\#DIVX\DIVX MOVIEZ\MadMov\Transcendance -Wal...,Matroska (MKV),NaN,Fichier multimédia,2.61 GB,movie/157353,2209764.0
3,3901,Shadow Warriors: Hattori Hanzo,影の軍団 服部半蔵 (Titre original),1980,133,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,Eiichi Kudo's strangest film that started the ...,"L'histoire se déroule vers 1650, après la mort...",F:\#DIVX\DIVX MOVIEZ\MadMov\Shadow Warriors - ...,Matroska (MKV),NaN,Fichier multimédia,996 MB,movie/208908,6975420.0
4,3900,Brainstorm,NaN,1983,106,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=NNiZP2G-nEM,... l'expérience ultime,"Lillian Reynolds et «Mike» Brace, réussissent ...",F:\#DIVX\DIVX MOVIEZ\MadMov\Brainstorm (1983) ...,Matroska (MKV),NaN,Fichier multimédia,5.92 GB,movie/15050,85271.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3899,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3900,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3901,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3902,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [414]:
display(df_intermediaire_movie_music)

,id,Numéro
0,0,3904
1,1,3903
2,1,3817
3,1,3520
4,1,3485
...,...,...
4336,1377,46
4337,1378,46
4338,1379,42
4339,1380,14


In [415]:
# Trouver l'index de la ligne avec le 'Compositeur(s)' NULL
null_music_index = df_music[df_music['Compositeur(s)'].isnull()].index

# Si une ligne avec 'Compositeur(s)' NULL est trouvée
if not null_music_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_music.loc[null_music_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_music['id'].max()
    if max_id > 0:
        df_music.loc[df_music.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_music.head())

# Enregistrer le DataFrame modifié dans la table 'music' de la base de données
df_music.to_sql("music", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_music = df_music.merge(df_music_movie, on="Compositeur(s)").drop(columns="Compositeur(s)")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_music.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_music.to_sql("movie_music", index=False, con=engine, if_exists="replace")


     id   Compositeur(s)
0  1382         Creation
1     1  Ennio Morricone
2     2    Mychael Danna
3     0              NaN
4     4     James Horner
     id  Numéro
0  1382    3904
1     1    3903
2     1    3817
3     1    3520
4     1    3485


4341

In [416]:
# df_music.to_sql("music",index=False,con = engine,if_exists = "replace" )

In [417]:
df_intermediaire_movie_music.to_sql("movie_music",index=False,con = engine,if_exists = "replace" )

4341

# table movie, studio et intermediaire

In [418]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [419]:
list_studio_movie = ["Studio", 'Numéro']
df_studio_movie = df.copy()[list_studio_movie]

In [420]:
df_studio_movie ["Studio"] = df_studio_movie["Studio"].str.split(",\s*")

In [421]:
df_studio_movie  = df_studio_movie.explode("Studio")

In [422]:
df_studio  = df_studio_movie.explode("Studio").drop_duplicates("Studio").reset_index(drop=True).reset_index()
df_studio = df_studio.rename(columns={"index":"id"}).drop(columns="Numéro")

In [423]:
df_intermediaire_movie_studio = df_studio.merge(df_studio_movie, on = "Studio").drop(columns="Studio")

In [424]:
display(df_studio)

,id,Studio
0,0,Toei Company
1,1,NaN
2,2,Alcon Entertainment
3,3,Metro-Goldwyn-Mayer
4,4,Central Motion Picture Corporation
...,...,...
1055,1055,ADR Distribution
1056,1056,A-Film Distribution
1057,1057,Dybex
1058,1058,British Empire Films Australia


In [425]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3904,Yokohama BJ blues (vostfr),ヨコハマＢＪブルース (Titre original),1981,112,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=oYzkD-YJbjo,NaN,BJ est un chanteur de blues relativement incon...,F:\#DIVX\DIVX MOVIEZ\POLAR\Yokohama BJ Blues (...,Matroska (MKV),NaN,Fichier multimédia,1.93 GB,movie/318955,83351.0
1,3903,Vertiges (vostfr),Per le antiche scale (Titre original),1975,110,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,Dante's Inferno is Alive and Well,"En Italie, dans les années 1930, le professeur...",F:\#DIVX\DIVX MOVIEZ\DRAMA\Vertiges (1975) VOS...,AVI (Microsoft),NaN,Fichier multimédia,1.73 GB,movie/105529,73529.0
2,3902,Transcendance,Transcendence (Titre original),2014,119,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=wUhsUgd-HrY,Hier le Dr Will Caster n'était qu'humain...,Le Dr Will Caster est un spécialiste de l'IA t...,F:\#DIVX\DIVX MOVIEZ\MadMov\Transcendance -Wal...,Matroska (MKV),NaN,Fichier multimédia,2.61 GB,movie/157353,2209764.0
3,3901,Shadow Warriors: Hattori Hanzo,影の軍団 服部半蔵 (Titre original),1980,133,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,Eiichi Kudo's strangest film that started the ...,"L'histoire se déroule vers 1650, après la mort...",F:\#DIVX\DIVX MOVIEZ\MadMov\Shadow Warriors - ...,Matroska (MKV),NaN,Fichier multimédia,996 MB,movie/208908,6975420.0
4,3900,Brainstorm,NaN,1983,106,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=NNiZP2G-nEM,... l'expérience ultime,"Lillian Reynolds et «Mike» Brace, réussissent ...",F:\#DIVX\DIVX MOVIEZ\MadMov\Brainstorm (1983) ...,Matroska (MKV),NaN,Fichier multimédia,5.92 GB,movie/15050,85271.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3899,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3900,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3901,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3902,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [426]:
display(df_intermediaire_movie_studio)

,id,Numéro
0,0,3904
1,0,3901
2,0,3889
3,0,3888
4,0,3854
...,...,...
3902,1055,182
3903,1056,172
3904,1057,169
3905,1058,108


In [427]:
# Trouver l'index de la ligne avec le 'Studio' NULL
null_studio_index = df_studio[df_studio['Studio'].isnull()].index

# Si une ligne avec 'Studio' NULL est trouvée
if not null_studio_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_studio.loc[null_studio_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_studio['id'].max()
    if max_id > 0:
        df_studio.loc[df_studio.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_studio.head())

# Enregistrer le DataFrame modifié dans la table 'studio' de la base de données
df_studio.to_sql("studio", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_studio = df_studio.merge(df_studio_movie, on="Studio").drop(columns="Studio")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_studio.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_studio.to_sql("movie_studio", index=False, con=engine, if_exists="replace")


     id                              Studio
0  1060                        Toei Company
1     0                                 NaN
2     2                 Alcon Entertainment
3     3                 Metro-Goldwyn-Mayer
4     4  Central Motion Picture Corporation
     id  Numéro
0  1060    3904
1  1060    3901
2  1060    3889
3  1060    3888
4  1060    3854


3907

In [428]:
# df_studio.to_sql("studio",index=False,con = engine,if_exists = "replace" )

In [429]:
df_intermediaire_movie_studio.to_sql("movie_studio",index=False,con = engine,if_exists = "replace" )

3907

# table movie, casting & intermediaire

In [430]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [431]:
list_casting_movie = ["Distribution", 'Numéro']
df_casting_movie = df.copy()[list_casting_movie]

In [432]:
# Nettoyer les noms d'acteurs en supprimant "(voix)" et autres parenthèses similaires
df_casting_movie["Distribution"] = df_casting_movie["Distribution"].str.replace(r"\s*\(.*?\)", "", regex=True)

In [433]:
df_casting_movie ["Distribution"] = df_casting_movie["Distribution"].str.split(",\s*")

In [434]:
df_casting_movie  = df_casting_movie.explode("Distribution")

In [435]:
df_casting  = df_casting_movie.explode("Distribution").drop_duplicates("Distribution").reset_index(drop=True).reset_index()
df_casting = df_casting.rename(columns={"index":"id"}).drop(columns="Numéro")

In [436]:
df_intermediaire_movie_casting = df_casting.merge(df_casting_movie, on = "Distribution").drop(columns="Distribution")

In [437]:
display(df_casting)

,id,Distribution
0,0,Yūsaku Matsuda
1,1,Mari Henmi
2,2,Keizo Kanie
3,3,Toby Kadoguchi
4,4,Haruko Mabuchi
...,...,...
10208,10208,Paulette Goddard
10209,10209,Henry Bergman
10210,10210,Tiny Sandford
10211,10211,Jackie Coogan


In [438]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3904,Yokohama BJ blues (vostfr),ヨコハマＢＪブルース (Titre original),1981,112,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=oYzkD-YJbjo,NaN,BJ est un chanteur de blues relativement incon...,F:\#DIVX\DIVX MOVIEZ\POLAR\Yokohama BJ Blues (...,Matroska (MKV),NaN,Fichier multimédia,1.93 GB,movie/318955,83351.0
1,3903,Vertiges (vostfr),Per le antiche scale (Titre original),1975,110,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,Dante's Inferno is Alive and Well,"En Italie, dans les années 1930, le professeur...",F:\#DIVX\DIVX MOVIEZ\DRAMA\Vertiges (1975) VOS...,AVI (Microsoft),NaN,Fichier multimédia,1.73 GB,movie/105529,73529.0
2,3902,Transcendance,Transcendence (Titre original),2014,119,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=wUhsUgd-HrY,Hier le Dr Will Caster n'était qu'humain...,Le Dr Will Caster est un spécialiste de l'IA t...,F:\#DIVX\DIVX MOVIEZ\MadMov\Transcendance -Wal...,Matroska (MKV),NaN,Fichier multimédia,2.61 GB,movie/157353,2209764.0
3,3901,Shadow Warriors: Hattori Hanzo,影の軍団 服部半蔵 (Titre original),1980,133,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,Eiichi Kudo's strangest film that started the ...,"L'histoire se déroule vers 1650, après la mort...",F:\#DIVX\DIVX MOVIEZ\MadMov\Shadow Warriors - ...,Matroska (MKV),NaN,Fichier multimédia,996 MB,movie/208908,6975420.0
4,3900,Brainstorm,NaN,1983,106,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=NNiZP2G-nEM,... l'expérience ultime,"Lillian Reynolds et «Mike» Brace, réussissent ...",F:\#DIVX\DIVX MOVIEZ\MadMov\Brainstorm (1983) ...,Matroska (MKV),NaN,Fichier multimédia,5.92 GB,movie/15050,85271.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3899,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3900,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3901,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3902,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [439]:
display(df_intermediaire_movie_casting)

,id,Numéro
0,0,3904
1,0,3824
2,0,3803
3,0,3684
4,0,3683
...,...,...
18504,10208,2
18505,10209,2
18506,10210,2
18507,10211,1


In [440]:
# Trouver l'index de la ligne avec la 'Distribution' NULL
null_casting_index = df_casting[df_casting['Distribution'].isnull()].index

# Si une ligne avec 'Distribution' NULL est trouvée
if not null_casting_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_casting.loc[null_casting_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_casting['id'].max()
    if max_id > 0:
        df_casting.loc[df_casting.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_casting.head())

# Enregistrer le DataFrame modifié dans la table 'casting' de la base de données
df_casting.to_sql("casting", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_casting = df_casting.merge(df_casting_movie, on="Distribution").drop(columns="Distribution")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_casting.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_casting.to_sql("movie_casting", index=False, con=engine, if_exists="replace")


      id    Distribution
0  10213  Yūsaku Matsuda
1      1      Mari Henmi
2      2     Keizo Kanie
3      3  Toby Kadoguchi
4      4  Haruko Mabuchi
      id  Numéro
0  10213    3904
1  10213    3824
2  10213    3803
3  10213    3684
4  10213    3683


18509

In [441]:
# df_casting.to_sql("casting",index=False,con = engine,if_exists = "replace" )

In [442]:
# df_intermediaire_movie_casting.to_sql("movie_casting",index=False,con = engine,if_exists = "replace" )

# table movie, country & intermediaire

In [443]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [444]:
list_country_movie = ["Pays", 'Numéro']
df_country_movie = df.copy()[list_country_movie]

In [445]:
df_country_movie ["Pays"] = df_country_movie["Pays"].str.split(",\s*")

In [446]:
df_country_movie  = df_country_movie.explode("Pays")

In [447]:
df_country  = df_country_movie.explode("Pays").drop_duplicates("Pays").reset_index(drop=True).reset_index()
df_country = df_country.rename(columns={"index":"id"}).drop(columns="Numéro")

In [448]:
df_intermediaire_movie_country = df_country.merge(df_country_movie, on = "Pays").drop(columns="Pays")

In [449]:
display(df_country)

,id,Pays
0,0,Japon
1,1,Italie
2,2,Etats-Unis
3,3,TW
4,4,France
5,5,Hong Kong
6,6,Suède
7,7,Autriche
8,8,Allemagne
9,9,Royaume-Uni


In [450]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3904,Yokohama BJ blues (vostfr),ヨコハマＢＪブルース (Titre original),1981,112,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=oYzkD-YJbjo,NaN,BJ est un chanteur de blues relativement incon...,F:\#DIVX\DIVX MOVIEZ\POLAR\Yokohama BJ Blues (...,Matroska (MKV),NaN,Fichier multimédia,1.93 GB,movie/318955,83351.0
1,3903,Vertiges (vostfr),Per le antiche scale (Titre original),1975,110,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,Dante's Inferno is Alive and Well,"En Italie, dans les années 1930, le professeur...",F:\#DIVX\DIVX MOVIEZ\DRAMA\Vertiges (1975) VOS...,AVI (Microsoft),NaN,Fichier multimédia,1.73 GB,movie/105529,73529.0
2,3902,Transcendance,Transcendence (Titre original),2014,119,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=wUhsUgd-HrY,Hier le Dr Will Caster n'était qu'humain...,Le Dr Will Caster est un spécialiste de l'IA t...,F:\#DIVX\DIVX MOVIEZ\MadMov\Transcendance -Wal...,Matroska (MKV),NaN,Fichier multimédia,2.61 GB,movie/157353,2209764.0
3,3901,Shadow Warriors: Hattori Hanzo,影の軍団 服部半蔵 (Titre original),1980,133,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,Eiichi Kudo's strangest film that started the ...,"L'histoire se déroule vers 1650, après la mort...",F:\#DIVX\DIVX MOVIEZ\MadMov\Shadow Warriors - ...,Matroska (MKV),NaN,Fichier multimédia,996 MB,movie/208908,6975420.0
4,3900,Brainstorm,NaN,1983,106,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=NNiZP2G-nEM,... l'expérience ultime,"Lillian Reynolds et «Mike» Brace, réussissent ...",F:\#DIVX\DIVX MOVIEZ\MadMov\Brainstorm (1983) ...,Matroska (MKV),NaN,Fichier multimédia,5.92 GB,movie/15050,85271.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3899,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3900,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3901,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3902,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [451]:
display(df_intermediaire_movie_country)

,id,Numéro
0,0,3904
1,0,3901
2,0,3889
3,0,3888
4,0,3887
...,...,...
3899,45,853
3900,46,921
3901,47,769
3902,48,631


In [452]:
# Trouver l'index de la ligne avec le 'Pays' NULL
null_country_index = df_country[df_country['Pays'].isnull()].index

# Si une ligne avec 'Pays' NULL est trouvée
if not null_country_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_country.loc[null_country_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_country['id'].max()
    if max_id > 0:
        df_country.loc[df_country.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_country.head())

# Enregistrer le DataFrame modifié dans la table 'country' de la base de données
df_country.to_sql("country", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_country = df_country.merge(df_country_movie, on="Pays").drop(columns="Pays")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_country.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_country.to_sql("movie_country", index=False, con=engine, if_exists="replace")

   id        Pays
0  49       Japon
1   1      Italie
2   2  Etats-Unis
3   3          TW
4   4      France
   id  Numéro
0  49    3904
1  49    3901
2  49    3889
3  49    3888
4  49    3887


3904

In [453]:
# df_country.to_sql("country",index=False,con = engine,if_exists = "replace" )

In [454]:
df_intermediaire_movie_country.to_sql("movie_country",index=False,con = engine,if_exists = "replace" )

3904

# table movie, genre & intermediaire

In [455]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [456]:
list_genre_movie = ["Genre(s)", 'Numéro']
df_genre_movie = df.copy()[list_genre_movie]

In [457]:
df_genre_movie ["Genre(s)"] = df_genre_movie["Genre(s)"].str.split(",\s*")

In [458]:
df_genre_movie  = df_genre_movie.explode("Genre(s)")

In [459]:
df_genre  = df_genre_movie.explode("Genre(s)").drop_duplicates("Genre(s)").reset_index(drop=True).reset_index()
df_genre = df_genre.rename(columns={"index":"id"}).drop(columns="Numéro")

In [460]:
df_intermediaire_movie_genre = df_genre.merge(df_genre_movie, on = "Genre(s)").drop(columns="Genre(s)")

In [461]:
display(df_genre)

,id,Genre(s)
0,0,Policier
1,1,Mystère
2,2,Drame
3,3,Suspense
4,4,Science-fiction
5,5,Action
6,6,Guerre
7,7,Comédie
8,8,Romantique
9,9,Western


In [462]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3904,Yokohama BJ blues (vostfr),ヨコハマＢＪブルース (Titre original),1981,112,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=oYzkD-YJbjo,NaN,BJ est un chanteur de blues relativement incon...,F:\#DIVX\DIVX MOVIEZ\POLAR\Yokohama BJ Blues (...,Matroska (MKV),NaN,Fichier multimédia,1.93 GB,movie/318955,83351.0
1,3903,Vertiges (vostfr),Per le antiche scale (Titre original),1975,110,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,Dante's Inferno is Alive and Well,"En Italie, dans les années 1930, le professeur...",F:\#DIVX\DIVX MOVIEZ\DRAMA\Vertiges (1975) VOS...,AVI (Microsoft),NaN,Fichier multimédia,1.73 GB,movie/105529,73529.0
2,3902,Transcendance,Transcendence (Titre original),2014,119,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=wUhsUgd-HrY,Hier le Dr Will Caster n'était qu'humain...,Le Dr Will Caster est un spécialiste de l'IA t...,F:\#DIVX\DIVX MOVIEZ\MadMov\Transcendance -Wal...,Matroska (MKV),NaN,Fichier multimédia,2.61 GB,movie/157353,2209764.0
3,3901,Shadow Warriors: Hattori Hanzo,影の軍団 服部半蔵 (Titre original),1980,133,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,Eiichi Kudo's strangest film that started the ...,"L'histoire se déroule vers 1650, après la mort...",F:\#DIVX\DIVX MOVIEZ\MadMov\Shadow Warriors - ...,Matroska (MKV),NaN,Fichier multimédia,996 MB,movie/208908,6975420.0
4,3900,Brainstorm,NaN,1983,106,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=NNiZP2G-nEM,... l'expérience ultime,"Lillian Reynolds et «Mike» Brace, réussissent ...",F:\#DIVX\DIVX MOVIEZ\MadMov\Brainstorm (1983) ...,Matroska (MKV),NaN,Fichier multimédia,5.92 GB,movie/15050,85271.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3899,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3900,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3901,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3902,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [463]:
display(df_intermediaire_movie_genre)

,id,Numéro
0,0,3904
1,0,3898
2,0,3892
3,0,3890
4,0,3889
...,...,...
8758,37,36
8759,37,35
8760,38,3113
8761,38,389


In [464]:
# Trouver l'index de la ligne avec le 'Genre(s)' NULL
null_genre_index = df_genre[df_genre['Genre(s)'].isnull()].index

# Si une ligne avec 'Genre(s)' NULL est trouvée
if not null_genre_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_genre.loc[null_genre_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_genre['id'].max()
    if max_id > 0:
        df_genre.loc[df_genre.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_genre.head())

# Enregistrer le DataFrame modifié dans la table 'genre' de la base de données
df_genre.to_sql("genre", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_genre = df_genre.merge(df_genre_movie, on="Genre(s)").drop(columns="Genre(s)")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_genre.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_genre.to_sql("movie_genre", index=False, con=engine, if_exists="replace")


   id         Genre(s)
0  38         Policier
1   1          Mystère
2   2            Drame
3   3         Suspense
4   4  Science-fiction
   id  Numéro
0  38    3904
1  38    3898
2  38    3892
3  38    3890
4  38    3889


8763

In [465]:
df_genre.to_sql("genre",index=False,con = engine,if_exists = "replace" )

39

In [466]:
df_intermediaire_movie_genre.to_sql("movie_genre",index=False,con = engine,if_exists = "replace" )

8763

# table movie, language & intermediaire

In [467]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [468]:
list_language_movie = ["Langues", 'Numéro']
df_language_movie = df.copy()[list_language_movie]

In [469]:
df_language_movie ["Langues"] = df_language_movie["Langues"].str.split(",\s*")

In [470]:
df_language_movie  = df_language_movie.explode("Langues")

In [471]:
df_language  = df_language_movie.explode("Langues").drop_duplicates("Langues").reset_index(drop=True).reset_index()
df_language = df_language.rename(columns={"index":"id"}).drop(columns="Numéro")

In [472]:
df_intermediaire_movie_language = df_language.merge(df_language_movie, on = "Langues").drop(columns="Langues")

In [473]:
display(df_language)

,id,Langues
0,0,Japonais
1,1,Italien
2,2,Anglais
3,3,Mandarin
4,4,Espagnol
5,5,Français
6,6,Cantonais
7,7,Danois
8,8,NaN
9,9,Film muet


In [474]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3904,Yokohama BJ blues (vostfr),ヨコハマＢＪブルース (Titre original),1981,112,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=oYzkD-YJbjo,NaN,BJ est un chanteur de blues relativement incon...,F:\#DIVX\DIVX MOVIEZ\POLAR\Yokohama BJ Blues (...,Matroska (MKV),NaN,Fichier multimédia,1.93 GB,movie/318955,83351.0
1,3903,Vertiges (vostfr),Per le antiche scale (Titre original),1975,110,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,Dante's Inferno is Alive and Well,"En Italie, dans les années 1930, le professeur...",F:\#DIVX\DIVX MOVIEZ\DRAMA\Vertiges (1975) VOS...,AVI (Microsoft),NaN,Fichier multimédia,1.73 GB,movie/105529,73529.0
2,3902,Transcendance,Transcendence (Titre original),2014,119,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=wUhsUgd-HrY,Hier le Dr Will Caster n'était qu'humain...,Le Dr Will Caster est un spécialiste de l'IA t...,F:\#DIVX\DIVX MOVIEZ\MadMov\Transcendance -Wal...,Matroska (MKV),NaN,Fichier multimédia,2.61 GB,movie/157353,2209764.0
3,3901,Shadow Warriors: Hattori Hanzo,影の軍団 服部半蔵 (Titre original),1980,133,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,Eiichi Kudo's strangest film that started the ...,"L'histoire se déroule vers 1650, après la mort...",F:\#DIVX\DIVX MOVIEZ\MadMov\Shadow Warriors - ...,Matroska (MKV),NaN,Fichier multimédia,996 MB,movie/208908,6975420.0
4,3900,Brainstorm,NaN,1983,106,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=NNiZP2G-nEM,... l'expérience ultime,"Lillian Reynolds et «Mike» Brace, réussissent ...",F:\#DIVX\DIVX MOVIEZ\MadMov\Brainstorm (1983) ...,Matroska (MKV),NaN,Fichier multimédia,5.92 GB,movie/15050,85271.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3899,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3900,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3901,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3902,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [475]:
display(df_intermediaire_movie_language)

,id,Numéro
0,0,3904
1,0,3901
2,0,3889
3,0,3888
4,0,3887
...,...,...
5465,43,1060
5466,44,1053
5467,44,930
5468,44,926


In [476]:
# Trouver l'index de la ligne avec les 'Langues' NULL
null_language_index = df_language[df_language['Langues'].isnull()].index

# Si une ligne avec 'Langues' NULL est trouvée
if not null_language_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_language.loc[null_language_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_language['id'].max()
    if max_id > 0:
        df_language.loc[df_language.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_language.head())

# Enregistrer le DataFrame modifié dans la table 'language' de la base de données
df_language.to_sql("language", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_language = df_language.merge(df_language_movie, on="Langues").drop(columns="Langues")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_language.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_language.to_sql("movie_language", index=False, con=engine, if_exists="replace")


   id   Langues
0  46  Japonais
1   1   Italien
2   2   Anglais
3   3  Mandarin
4   4  Espagnol
   id  Numéro
0  46    3904
1  46    3901
2  46    3889
3  46    3888
4  46    3887


5470

In [477]:
# df_language.to_sql("language",index=False,con = engine,if_exists = "replace" )

In [478]:
df_intermediaire_movie_language.to_sql("movie_language",index=False,con = engine,if_exists = "replace" )

5470

# table, movie, tag & intermediaire

In [479]:
# list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
#                    'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
#                    'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
#                    'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
# df_movie = df.copy()[list_col_movies]

In [480]:
# list_tag_movie = ["Etiquettes", 'Numéro']
# df_tag_movie = df.copy()[list_tag_movie]

In [481]:
# df_tag_movie ["Etiquettes"] = df_tag_movie["Etiquettes"].str.split(",\s*")

In [482]:
# df_tag_movie  = df_tag_movie.explode("Etiquettes")

In [483]:
# df_tag = df_tag_movie.explode("Etiquettes").drop_duplicates("Etiquettes").reset_index(drop=True).reset_index()
# df_tag = df_tag.rename(columns={"index":"id"}).drop(columns="Numéro")

In [484]:
# df_intermediaire_movie_tag = df_tag.merge(df_tag_movie, on = "Etiquettes").drop(columns="Etiquettes")

In [485]:
# display(df_tag)

In [486]:
# display(df_movie)

In [487]:
# display(df_intermediaire_movie_tag)

In [488]:
# # Trouver l'index de la ligne avec les 'Etiquettes' NULL
# null_tag_index = df_tag[df_tag['Etiquettes'].isnull()].index

# # Si une ligne avec 'Etiquettes' NULL est trouvée
# if not null_tag_index.empty:
#     # Attribuer à cette ligne l'ID 0
#     df_tag.loc[null_tag_index, 'id'] = 0

#     # Décaler l'ID de la première ligne (anciennement index 0)
#     max_id = df_tag['id'].max()
#     if max_id > 0:
#         df_tag.loc[df_tag.index[0], 'id'] = max_id + 1

# # Afficher le DataFrame modifié
# print(df_tag.head())

# # Enregistrer le DataFrame modifié dans la table 'tag' de la base de données
# df_tag.to_sql("tag", index=False, con=engine, if_exists="replace")

# # Recréer la table intermédiaire avec les données mises à jour
# df_intermediaire_movie_tag = df_tag.merge(df_tag_movie, on="Etiquettes").drop(columns="Etiquettes")

# # Afficher la table intermédiaire mise à jour
# print(df_intermediaire_movie_tag.head())

# # Enregistrer la table intermédiaire mise à jour dans la base de données
# df_intermediaire_movie_tag.to_sql("movie_tag", index=False, con=engine, if_exists="replace")


In [489]:
# df_tag.to_sql("tag",index=False,con = engine,if_exists = "replace" )

In [490]:
# df_intermediaire_movie_tag.to_sql("movie_tag",index=False,con = engine,if_exists = "replace" )